In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive

In [ ]:
!ls

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
movies=pd.read_csv('movies_genres.csv')

In [ ]:
data=pd.read_excel('감성대화말뭉치(최종데이터)_Training.xlsx')

In [ ]:
data=data.iloc[:,5:8]

In [ ]:
data.drop('감정_소분류', axis=1, inplace=True)

In [ ]:
data.columns=['감정', '사람']
# data

In [ ]:
data.head()

,감정,사람
0,기쁨,아내가 드디어 출산하게 되어서 정말 신이 나.
1,불안,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.
2,당황,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.
3,기쁨,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.
4,기쁨,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.


In [ ]:
data=data.replace('불안', 0).replace('분노', 1).replace('슬픔', 2).replace('기쁨', 3).replace('상처', 4).replace('당황', 5)

In [ ]:
# data.감정.unique()
data=data.replace('불안 ', 0).replace('기쁨 ', 1)

In [ ]:
data.감정.unique()

array([3, 0, 5, 2, 1, 4])

In [ ]:
data.head()

,감정,사람
0,3,아내가 드디어 출산하게 되어서 정말 신이 나.
1,0,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.
2,5,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.
3,3,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.
4,3,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.


In [ ]:
!pip install mxnet
!pip install gluonnlp
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 1.2 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 31.7 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp38-cp38-linux_x86_64.whl size=619633 sha256=87b6b3b6cd018d33bed7ae0ad6da529cb4e6c128a47eef3dc13ef6005c0a9113
  Stored in directory: /root/.cache/pip/wheels/b6/93/9d/2237550c409eb3ed725d6302b7897ddd9a037b40cef66dcd9c
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-wceldnob
  Running command

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm

In [ ]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
max_len = 64   # 텍스트 데이터 최대 길이
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
data_list = []
for ques, label in zip(data['사람'], data['감정'])  :
    data = []   
    data.append(ques)
    data.append(str(label))
    data_list.append(data)

In [ ]:
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=34)

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6, 
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(),
                              attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 대표적인 loss func

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 0/511 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
  0%|          | 1/511 [00:01<08:33,  1.01s/it]

epoch 1 batch id 1 loss 1.708181381225586 train acc 0.25


 39%|███▉      | 201/511 [02:08<03:17,  1.57it/s]

epoch 1 batch id 201 loss 1.1568039655685425 train acc 0.48880597014925375


 78%|███████▊  | 401/511 [04:16<01:11,  1.54it/s]

...

epoch 5 batch id 401 loss 0.4694223999977112 train acc 0.8040445760598504


100%|██████████| 511/511 [05:29<00:00,  1.55it/s]


epoch 5 train acc 0.8069325047370547


100%|██████████| 128/128 [00:28<00:00,  4.51it/s]

epoch 5 test acc 0.6515706380208334


In [ ]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=6)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        test_eval=[]
        re_movie=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("불안한")
                re_movie.append(movies.제목[movies['영화'] == np.argmax(logits)].to_list())
            elif np.argmax(logits) == 1:
                test_eval.append("분노가 치미는")
                re_movie.append(movies.제목[movies['영화'] == np.argmax(logits)].to_list())
            elif np.argmax(logits) == 2:
                test_eval.append("슬픈")
                re_movie.append(movies.제목[movies['영화'] == np.argmax(logits)].to_list())
            elif np.argmax(logits) == 3:
                test_eval.append("기쁜")
                re_movie.append(movies.제목[movies['영화'] == np.argmax(logits)].to_list())
            elif np.argmax(logits) == 4:
                test_eval.append("당황스러운")
                re_movie.append(movies.제목[movies['영화'] == np.argmax(logits)].to_list())
            elif np.argmax(logits) == 5:
                test_eval.append("상처 받는")
                re_movie.append(movies.제목[movies['영화'] == np.argmax(logits)].to_list())

        print("오늘 " + test_eval[0] + " 일이 있었군요")
        
        ans=input("오늘 기분에 맞는 영화를 추천해드릴까요? ")
        
        if ans == '응' :
          print("오늘 추천 드릴 영화는 " + re_movie[0][0] +", " +re_movie[0][1] +", " +re_movie[0][2]  + " 입니다.")
        else :
          print("좋은 하루 되세요")


In [ ]:
end = 1
while end == 1 :
    sentence = input("오늘 있었던 일을 입력해주세요 : ")
    if sentence == "그만" :
        break
    predict(sentence)
    print("\n")

오늘 있었던 일을 입력해주세요 : 오늘 친구랑 맛있는 음식을 먹었어
오늘 기쁜 일이 있었군요
오늘 기분에 맞는 영화를 추천해드릴까요? 응
오늘 추천 드릴 영화는 스파이 지니어스, 온워드: 단 하루의 기적, 겨울왕국 2 입니다.


오늘 있었던 일을 입력해주세요 : 어제 밤에 지인이랑 싸웠어
오늘 당황스러운 일이 있었군요
오늘 기분에 맞는 영화를 추천해드릴까요? 아니
좋은 하루 되세요


오늘 있었던 일을 입력해주세요 : 밤새 코드 짠 거를 저장도 못하고 날렸어
오늘 분노가 치미는 일이 있었군요
오늘 기분에 맞는 영화를 추천해드릴까요? 응
오늘 추천 드릴 영화는 반도, 테넷, 미드웨이 입니다.


오늘 있었던 일을 입력해주세요 : 그만
